In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

#  Worldometer somehow blocked me scraping their data ^^^

so the next few cells are now irrelevant, but no harm in keeping them.

(actually have deleted them now but want to remember this happened)

# Wikipedia data

For scraping the data from wikipedia for the rest of the world (ROW), source https://en.wikipedia.org/wiki/Template:2019–20_coronavirus_pandemic_data/International_medical_cases#Table, I resorted to copying and pasting the data into an excel spreadsheet 'myfile.xlsx' sheet 'world-china'


For scraping the data from wikipedia for China, source https://en.wikipedia.org/wiki/Template:2019–20_coronavirus_pandemic_data/China_medical_cases_by_province, copied to 'myfile.xlsx' sheet 'china'.



In [3]:
df_withoutChina = pd.read_excel (r'myfile.xlsx',sheet_name='world-china')
df_china = pd.read_excel (r'myfile.xlsx',sheet_name='china')
df_withoutChina_deaths = pd.read_excel (r'myfile.xlsx',sheet_name='world-china-deaths')



# Neatening the numpy arrays of data

Data is from table with regular titles at intervals, such as 'Date' repetition (for viewing purposes on wikipedia)

Had to remove this for ease of analysis.

In [4]:
def remove_annoying_rows(array):
    counter = 0
    ignore_rows=[0,1,2]
    while counter < np.shape(array)[0]:
        row = array[counter]
        is_row_string = False
        if counter in ignore_rows:
            pass
        else:
            if type(row[0]) == datetime:
                pass
            elif str(row[0])[0] == '2':
                    pass
            elif type(row[0]) == pd.Timestamp:
                pass
            else:
                is_row_string = True
            
            if is_row_string == True:
                array = np.delete(array, (counter), axis=0)
        counter +=1 
    
    return(array)
numpy_tables_withoutChina = remove_annoying_rows(pd.DataFrame.to_numpy(df_withoutChina,dtype='object'))
numpy_tables_china = remove_annoying_rows(pd.DataFrame.to_numpy(df_china,dtype='object'))
numpy_tables_withoutChina_deaths = remove_annoying_rows(pd.DataFrame.to_numpy(df_withoutChina_deaths,dtype='object'))


# Create dictionaries for datasets 

to help with indexing down the line

In [5]:
def create_dictionaryOf_indexes(data):
    dictionary = {}
    for i,country in enumerate(data[0]):
        if country == '(local)':
            pass
        elif country == 'New that day' or country == 'Sum to date':
            pass
        else:
            dictionary[str(country)] = i
    return(dictionary)

dictionary_cases_index = create_dictionaryOf_indexes(numpy_tables_withoutChina)
dictionary_deaths_index = create_dictionaryOf_indexes(numpy_tables_withoutChina_deaths)



In [6]:
#create a list of problamatic countries whose names are not the same on the cases spreadsheet and deaths spreadsheet
problomatic_countries = []
list_countries_cases = []
list_countries_deaths = []

for country in (numpy_tables_withoutChina[0]): list_countries_cases.append(country)
for country in numpy_tables_withoutChina_deaths[0]: 
    list_countries_deaths.append(country)
    if country not in list_countries_cases:
        problomatic_countries.append(country)

for country in numpy_tables_withoutChina[0]: 
    if country not in list_countries_deaths:
        problomatic_countries.append(country)


#  Data for China

China has a seperate wikipedia page therefore have to calculate values seperately

In [7]:
yvals_china = []
dates_china = []
previous_date_point = 0
number_cases = 0
date=0

for date_point in numpy_tables_china[3:]:
    date = date_point[0]
    if type(date) != str and type(date_point[32]) != float:
        number_cases = date_point[32] + previous_date_point
        previous_date_point = number_cases

        dates_china.append(date)
        yvals_china.append(number_cases)
            
###need to convert dates_china from timestamp type to datetime tipe for analysis later
for i,date in enumerate(dates_china):
    if type(date) == pd.Timestamp:
        dates_china[i] = date.to_pydatetime()
    else:
        print(type(date))


# Data for Rest of World


In [8]:
def produce_cumulative_data(array):
    '''produce numpy array with structure:
            date1 date2 date3 date4 date5 ... dateToday
    country1: 0     0     1     2     5
    country2: 0     1     7     19    60  etc.
    country3:
    ...
    countryN
    '''
    
    
    index_length = np.shape(array)[1]
    previous_cases = np.zeros(shape = (np.shape(array)[1]),dtype=int)
    dates_ = np.zeros(shape=(1,np.shape(array)[0]),dtype = 'object')
    yvals_ = np.zeros(shape=(np.shape(array)),dtype = 'object').T
    

    for i,date_point in enumerate(array):
        if type(date_point[0]) != float:
            if date_point[0][0] == '2':
                for letter in range(len(date_point[0])):
                    if date_point[0][letter] == '.': 
                        date_point[0] = date_point[0][:letter]+'-'+date_point[0][letter+1:]
                for letter in range(1,len(date_point[0])-1):
                    if date_point[0][letter-1] == '-' and date_point[0][letter+1] == '-':
                        date_point[0] = date_point[0][:letter] + '0' + date_point[0][letter:]

                date = datetime.strptime(date_point[0], '%y-%m-%d')
                number_cases = np.zeros((len(array[0])))
                for j in range(index_length):
                    if type(date_point[j]) == int:
                        if type(previous_cases[i]) == np.int32:
                            number_cases[j] = int(date_point[j] + previous_cases[j])
                            previous_cases[j] = int(number_cases[j])

                        elif previous_cases[j] == 0:
                            number_cases[j] = date_point[j]
                            previous_cases[j] = int(number_cases[j])
                    else:
                        number_cases[j] = previous_cases[j]
                    yvals_[j][i] = number_cases[j]                                            
                dates_[0][i] = date 
    dates_ = dates_[0]
    
    columns_to_be_deleted = []
    for j,day in enumerate(dates_):
        if day == 0:
            columns_to_be_deleted.append(j)

    dates_ = np.delete(dates_,columns_to_be_deleted,0)
    yvals_ = np.delete(yvals_,columns_to_be_deleted,1)
    print('date range :', dates_[0],'to: ',dates_[-1])

    
    return(dates_,yvals_)
                
dates_cases,yvals_cases = produce_cumulative_data(numpy_tables_withoutChina)


date range : 2020-01-13 00:00:00 to:  2020-03-31 00:00:00


# Fatality data for ROW

In [9]:
dates_deaths,yvals_deaths = produce_cumulative_data(numpy_tables_withoutChina_deaths)

date range : 2020-02-01 00:00:00 to:  2020-03-31 00:00:00


# Fatality rate data for ROW

In [10]:
def find_index_forDate(date_to_be_indexed,cases_or_deaths):
    '''finds index for given date'''
    
    if cases_or_deaths == 'cases':
        for i,date in enumerate(dates_cases):
            if date == date_to_be_indexed:
                return(i)
    elif cases_or_deaths == 'deaths':
        for i,date in enumerate(dates_deaths):
            if date == date_to_be_indexed:
                return(i)
            

def find_countryName_fromDeathIndex(index_in_deaths):
    for country_name in dictionary_deaths_index:
#         print(dictionary_deaths_index.get(country_name),index_in_deaths)
        if dictionary_deaths_index.get(country_name) == index_in_deaths:
            return(country_name)

        
def produce_fatality_rate(percentage = True):
    '''creates array with same dimensions as deaths array'''
    fatality_rates = np.zeros(shape=(np.shape(yvals_deaths)),dtype = float)
    
    
    for i,country in enumerate(yvals_deaths):
        if find_countryName_fromDeathIndex(i) is not None:
            country_string = find_countryName_fromDeathIndex(i)
            if country_string in dictionary_deaths_index and country_string in dictionary_cases_index:
                index_for_casesArray = dictionary_cases_index.get(country_string)
                for j,date in enumerate(dates_deaths):
                    index_fordate_inCases = find_index_forDate(date,'cases') #probably couldve done this earlier on to prevent it being done everytime this function iterates for each country
                    if yvals_cases[index_for_casesArray,index_fordate_inCases] != 0:
                        fatality_rates[i,j] = 100*yvals_deaths[i,j]/yvals_cases[index_for_casesArray,index_fordate_inCases]
    return(fatality_rates)          

yvals_fatality_rates = produce_fatality_rate()

# Differentiation algorithm

In [35]:
def differentiate(yvalues,time_periodInDays=1):
    yvals = np.zeros(shape=(np.shape(yvalues)),dtype = float)
    for i,country in enumerate(yvalues):
        for j,value_on_date in enumerate(country):
            if j != 0:   
                yvals[i,j] = (yvalues[i,j] - yvalues[i,j-1])/time_periodInDays
    return(yvals)

# Exponential algorithm

In [46]:
def exponential(yvalues):
    yvals = np.zeros(shape=(np.shape(yvalues)),dtype = float)
    for i,country in enumerate(yvalues):
        for j,value_on_date in enumerate(country):
            if j!= 0:
                yvals[i,j] = (yvalues[i,j] - yvalues[i,j-1])/yvalues[i,j]
    return(yvals)

# Plotting the data

In [56]:
import matplotlib as lib
import matplotlib.dates as mdates
import matplotlib.cbook as cbook


def plot(type_of_plot,countries_list,cases_or_deaths,date_min_index = 0, date_max_index = -1, timeshift = None):
    '''Different types of plot are: cumulative, new, acceleration,exponential,fatality rate.  
    timeshift is in days and always applies to UK'''
    
    #  check of input
    assert(type(countries_list) == list)
    for country in countries_list:
        assert(country not in problomatic_countries), 'The country ' + str(country) + 'is problomatic'
        if cases_or_deaths =='cases':
            assert(country in dictionary_cases_index), str(country) + ' is not in ' + str(dictionary_cases_index)
        elif cases_or_deaths == 'deaths':
            assert(country in dictionary_deaths_index), str(country) + ' is not in ' + str(dictionary_deaths_index)
    if type_of_plot == 'fatality rate': assert(cases_or_deaths == None), 'Fatality rate requires argument "cases_or_deaths_" = None, but currently inputed as: ' + str(cases_or_deaths)
     
    #  setup stuff
    fig, ax = plt.subplots(constrained_layout=True)
    locator = mdates.AutoDateLocator()
    formatter = mdates.ConciseDateFormatter(locator)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)

    ax.grid(True)
    if type_of_plot  == 'fatality rate':
        ax.title.set_text(str(type_of_plot))
        ax.set_ylabel('percentage')
    elif timeshift is not None:
        ax.title.set_text(str(type_of_plot) + ' plot of ' + str(cases_or_deaths) + ' with UK shifted back by ' +str(timeshift) + 'days')
    elif type_of_plot == 'exponential':
        ax.set_ylabel('Time Constant')
        ax.title.set_text(str(type_of_plot) + ' plot of ' + str(cases_or_deaths))
    else:
        ax.title.set_text(str(type_of_plot) + ' plot of ' + str(cases_or_deaths))
        
    
    #  define dates and yvals for each case
    if cases_or_deaths == 'cases':
        if type_of_plot == 'cumulative':
            dates,yvals = dates_cases,yvals_cases
        elif type_of_plot == 'new':
            dates,yvals = dates_cases,differentiate(yvals_cases)
        elif type_of_plot == 'acceleration':
            dates,yvals = dates_cases,differentiate(differentiate(yvals_cases))
        elif type_of_plot == 'exponential':
            dates,yvals = dates_cases,exponential(yvals_cases)
        else:
            raise ValueError('type_of_plot is not inputed correctly')
    
    elif cases_or_deaths == 'deaths':
        if type_of_plot == 'cumulative':
            dates,yvals = dates_deaths,yvals_deaths
        elif type_of_plot == 'new':
            dates,yvals = dates_deaths,differentiate(yvals_deaths)
        elif type_of_plot == 'acceleration':
            dates,yvals = dates_deaths,differentiate(differentiate(yvals_deaths))
        elif type_of_plot == 'exponential':
            dates,yvals = dates_deaths,exponential(yvals_deaths)
        else:
            raise ValueError('type_of_plot is not inputed correctly')
    
    elif type_of_plot == 'fatality rate':
        dates,yvals = dates_deaths,yvals_fatality_rates
        
    else:
        raise ValueError('cases or deaths is not inputed correctly')
    
    #  now for timeshift effect
    if timeshift is not None:
        assert(type(timeshift) == int), 'timeshift needs to be in integers of days, current timeshift is '+ str(timeshift) + ' and type: ' + str(type(timeshift))
        altered_dates_uk = []
        for date in dates:
            altered_dates_uk.append(np.datetime64(date)-np.timedelta64(timeshift, 'D'))
        
    
    for country in countries_list:    
        if cases_or_deaths == 'cases':
            if country == 'United Kingdom':
                if timeshift is not None:
                    ax.plot(altered_dates_uk,yvals[dictionary_cases_index.get('United Kingdom')],label = ' Shifted UK',color = 'r')
                else:
                    ax.plot(dates,yvals[dictionary_cases_index.get('United Kingdom')],label = 'UK',color = 'r') # I wanted the UK to always be red
            elif country == 'Spain':
                ax.plot(dates,yvals[dictionary_cases_index.get('Spain')],label = 'Spain',color = 'y') # I wanted Spain to always be yellow
            elif country == 'Italy':
                ax.plot(dates,yvals[dictionary_cases_index.get('Italy')],label = 'Italy',color = 'g') # I wanted Italy to always be green       
            elif country == 'United States':
                ax.plot(dates,yvals[dictionary_cases_index.get('United States')],label = 'USA',color = 'b') # I wanted USA to always be blue
            else:
                ax.plot(dates,yvals[dictionary_cases_index.get(country)],label = str(country))        
        elif cases_or_deaths == 'deaths':
            if country == 'United Kingdom':
                if timeshift is not None:
                    ax.plot(altered_dates_uk,yvals[dictionary_deaths_index.get('United Kingdom')],label = ' Shifted UK',color = 'r')
                else:
                    ax.plot(dates,yvals[dictionary_deaths_index.get('United Kingdom')],label = 'UK',color = 'r') # I wanted the UK to always be red
            elif country == 'Spain':
                ax.plot(dates,yvals[dictionary_deaths_index.get('Spain')],label = 'Spain',color = 'y') # I wanted Spain to always be yellow
            elif country == 'Italy':
                ax.plot(dates,yvals[dictionary_deaths_index.get('Italy')],label = 'Italy',color = 'g') # I wanted Italy to always be green       
            elif country == 'United States':
                ax.plot(dates,yvals[dictionary_deaths_index.get('United States')],label = 'USA',color = 'b') # I wanted USA to always be blue
            else:
                ax.plot(dates,yvals[dictionary_deaths_index.get(country)],label = str(country))
        elif type_of_plot == 'fatality rate':
            if country == 'United Kingdom':
                if timeshift is not None:
                    ax.plot(altered_dates_uk,yvals[dictionary_deaths_index.get('United Kingdom')],label = ' Shifted UK',color = 'r')
                else:
                    ax.plot(dates,yvals[dictionary_deaths_index.get('United Kingdom')],label = 'UK',color = 'r') # I wanted the UK to always be red
            elif country == 'Spain':
                ax.plot(dates,yvals[dictionary_deaths_index.get('Spain')],label = 'Spain',color = 'y') # I wanted Spain to always be yellow
            elif country == 'Italy':
                ax.plot(dates,yvals[dictionary_deaths_index.get('Italy')],label = 'Italy',color = 'g') # I wanted Italy to always be green       
            elif country == 'United States':
                ax.plot(dates,yvals[dictionary_deaths_index.get('United States')],label = 'USA',color = 'b') # I wanted USA to always be blue
            else:
                ax.plot(dates,yvals[dictionary_deaths_index.get(country)],label = str(country))
    
    datemin = np.datetime64(dates[date_min_index], 'D')
    datemax = np.datetime64(dates[date_max_index], 'D')
    ax.set_xlim(datemin, datemax)

    ax.legend(loc='best')
    
    
    fig.savefig('plots/' + str(type_of_plot) + str(cases_or_deaths) + str(countries_list) + 'timeshift='+str(timeshift)+'.png')
    



In [58]:
# plot('cumulative',['United Kingdom','United States','Italy','Spain','France'],'cases',date_min_index=40)
# plot('cumulative',['Italy','Spain'],'deaths',date_min_index=18)
# plot('cumulative',['United Kingdom','France'],'cases',timeshift = 7,date_min_index=40)
# plot('fatality rate',['United Kingdom','Italy','Spain'],cases_or_deaths=None)
# plot('acceleration',['Italy','Spain'],'deaths',date_min_index=15)
# plot('exponential',['United Kingdom','Italy','France','Spain'],'cases',date_min_index=20)
# plot('exponential',['United Kingdom','Italy','France','Spain'],'deaths')


# My First Theory after looking at the data

The above chart seems to show a trend that every day, the number of new cases,N ~ 30% (R) of total cases over 10 days previously.

\begin{equation*}
    N(X) = \left(R \sum_{x=-10}^{0}N(x) \right) 
\end{equation*}
where k is in days

In [15]:
yvals_governement_effect[uk_index][45:]

NameError: name 'yvals_governement_effect' is not defined

In [ ]:
#start from first value:
R = np.mean(yvals_governement_effect[uk_index][47:])/100
print(R)

In [ ]:
predictions = np.zeros(shape =(14,2),dtype='object')

for i,row in enumerate(predictions):
    row[0] = np.datetime64(dates[-10], 'D') + np.timedelta64(i, 'D')
    row[1] = yvals_[uk_index][-10+i] 
    if i == 10:
        row[1] += R*(yvals_[uk_index][-1]- yvals_[uk_index][-10]) + predictions[i-1][1]
    elif i > 10:
        row[1] += R*(predictions[i-1][1] - yvals_[uk_index][i-20]) + predictions[i-1][1]


predictions